In [1]:
# Required imports
import json
import collections  # From Python standard library.
import bson
from bson.codec_options import CodecOptions
import pandas as pd
import datetime
import seaborn as sns

In [2]:
# Use json.load to read json encoded file and convert it to a Python dictionary
api_specs = json.load(open("/Users/deepanshachowdhary/Documents/thesis prac/final/api_specs.json", encoding = "utf-8"))
commits = json.load(open("/Users/deepanshachowdhary/Documents/thesis prac/final/commits.json", encoding = "utf-8"))

In [3]:
data = []  # define empty list to append all columns later on

In [4]:
# Match "api_spec_id" in commits file to "id" in specs file, then consider only valid files, and map to the empty data list defined above.

for spec in api_specs:
    for commit in commits:
        if commit["valid"] == 0:
            continue
        # if commit["id"] > spec["id"]:
        #     break
        if spec["id"] == commit["api_spec_id"]:
            externalDesc=""
            if "externalDocs" in commit["api_spec"]:
                if commit["api_spec"]["externalDocs"] != None:
                    if "description" in commit["api_spec"]["externalDocs"]:
                        externalDesc = commit["api_spec"]["externalDocs"]["description"]
            
            server_description=[]
            if "servers" in commit["api_spec"]:
                if commit["api_spec"]["servers"] != None:
                    for entry in commit["api_spec"]["servers"]:
                     if "description" in entry:
                        server_description.append(entry["description"])
            
            server_urls=[]
            if "servers" in commit["api_spec"]:
                if commit["api_spec"]["servers"] != None:
                    for entry in commit["api_spec"]["servers"]:
                        if "url" in entry:
                            server_urls.append(entry["url"])

            info_title=""
            if "info" in commit["api_spec"]:
                if commit["api_spec"]["info"] != None:
                    if "title" in commit["api_spec"]["info"]:
                        info_title = commit["api_spec"]["info"]["title"]

            info_version=""
            if "info" in commit["api_spec"]:
                if commit["api_spec"]["info"] != None:
                    if "version" in commit["api_spec"]["info"]:
                        info_version = commit["api_spec"]["info"]["version"]

            info_description=""
            if "info" in commit["api_spec"]:
                if commit["api_spec"]["info"] != None:
                    if "description" in commit["api_spec"]["info"]:
                        info_description = commit["api_spec"]["info"]["description"]

            processed_at = ""
            if "processed_at" in commit:
                if commit["processed_at"]!=None:
                    if "$date" in commit["processed_at"]:
                        if commit["processed_at"]["$date"]!= None:
                            if "$numberLong" in commit["processed_at"]["$date"]:
                                processed_at = commit["processed_at"]["$date"]["$numberLong"]
            data.append([spec["id"], spec["repo_name"], spec["owner"], spec["filename"], spec["commits"],
            commit["commit_date"]["$date"]["$numberLong"], commit["api_spec_id"], commit["api_version"],
            commit["api_title"],
            processed_at,
            commit["api_spec"]["openapi"] if "openapi" in commit["api_spec"] else None,
            info_description,
            info_version,
            info_title,
            externalDesc,
            server_description,
            server_urls
            ])


In [5]:
# Define dataframe with all the required columns
df_history = pd.DataFrame(data,columns=['id','repo_name','owner', 'filename', 'commits', 'commitDate', 'api_spec_id','api_version','api_title','processed_at',
'openapi','info_description','info_version','info_title','externalDocs','servers_description','servers_urls' ])

In [6]:
# Drop all missing values, in rows and columns both
api_history = df_history.dropna(axis=1,how='all')
display(api_history)

,id,repo_name,owner,filename,commits,commitDate,api_spec_id,api_version,api_title,processed_at,openapi,info_description,info_version,info_title,externalDocs,servers_description,servers_urls
0,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1617799976000,84,1.1.0,SQAaaS API,1639562016000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/..."
1,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1622430757000,84,1.1.0,SQAaaS API,1639562021000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/..."
2,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1624258233000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/..."
3,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1623993083000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/..."
4,124,challenge-schemas,Sage-Bionetworks,openapi.yaml,37,1607286945000,124,0.1.0,Registry of Open Community Challenge API,1639561989000,3.0.2,The OpenAPI specification implemented by the C...,0.1.0,Registry of Open Community Challenge API,,[],[{protocol}://rocc.com/{environment}/v1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67225,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1578550863000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.8,Avi TestSeDatastoreLevel2 Object API,,[],[]
67226,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1574229808000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.7,Avi TestSeDatastoreLevel2 Object API,,[],[]
67227,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1573786228000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,20.1.1,Avi TestSeDatastoreLevel2 Object API,,[],[]
67228,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1569918516000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.6,Avi TestSeDatastoreLevel2 Object API,,[],[]


In [7]:
# Convert column server_urls into a list for better viewing
server_list = api_history["servers_urls"].values.tolist()
server_list = [x for x in server_list if x != []]
# print(server_list)

In [8]:
api_history["info_version"] = api_history["info_version"].astype("category") # convert the version column to a categorical variable

In [9]:
print(api_history["info_version"].cat.categories) #print the categories

Index(['', '${VERSION}', '${git.build.version}', '${project.version}', '-',
       '-SNAPSHOT', '0', '0.0', '0.0-oas3', '0.0.0',
       ...
       'v3', 'v4', 'v4beta1', 'v7.75.0', 'v7.78.1', 'version not set',
       '{VERSION}', '{{ version }}', '“1.3.0”', 'バージョン(1.0)'],
      dtype='object', length=2802)


In [10]:
col_list = api_history.info_version.values.tolist()
# print(col_list)

In [11]:
# Define set of versions for all the apis
import numpy as np

df1 = pd.DataFrame({'Names':[['develop', 'draft', 'genesis', 'living', 'alpha', 'master', 'next', 'BETA', 'DEV', 'VERSION' , 'ALL','main','${VERSION}','{{ version }}', 'dev', 'Beta','${project.version}','unsupported','<overriden by annotation see Application class>']],
                    'SemVer7': [['7.0', '7.0.0','7.1.0', '7.1.1', '7.10.4', '7.10.5', '7.18.0']],
                    'SemVer6': [['6.0.0', '6.0.1', '6.0.2', '6.0.3', '6.1.0', '6.1.0-beta4', '6.1.1', '6.1.2', '6.1.3', '6.1.4', '6.2.0', '6.2.1', '6.6']],
                    'SemVer5': [['5.0.0', '5.0.0-beta1', '5.0.1', '5.1.0', '5.1.0-beta2', '5.10.0', '5.11.0', '5.12.0', '5.12.1', '5.12.2', '5.13.0', '5.14.0', '5.15.0', '5.15.1', '5.16.0', '5.16.1', '5.17.0', '5.18.0', '5.19.0', '5.2.0', '5.2.1', '5.20.0', '5.21.0', '5.22.0', '5.23.0', '5.24.0', '5.25.0', '5.26.0', '5.27.0', '5.27.1', '5.28.0', '5.28.1', '5.28.2', '5.28.3', '5.3.0', '5.3.1', '5.3.2', '5.3.3', '5.4.0', '5.4.1', '5.4.1-beta1', '5.4.2', '5.5.0', '5.5.0-beta1', '5.5.1', '5.5.2', '5.5.3', '5.5.4', '5.5.5', '5.5.6', '5.6.0', '5.6.1', '5.7.0', '5.8.0', '5.8.1', '5.9.1']],
                    'SemVer4': [['4.0', '4.0.0', '4.0.0-RC3', '4.0.0-beta', '4.0.0-beta-2', '4.0.0-pp', '4.0.0-rc1', '4.0.0-rc3', '4.0.0-rc4', '4.0.1', '4.0.2', '4.0.3', '4.0.4', '4.0.9', '4.0.90', '4.1.0', '4.1.0.0', '4.1.1', '4.1.10', '4.1.11', '4.1.12', '4.1.2', '4.1.3', '4.1.4', '4.1.5', '4.1.6', '4.1.7', '4.1.8', '4.1.9', '4.2.0', '4.2.1', '4.2.10', '4.2.11', '4.2.12', '4.2.2', '4.2.2.0', '4.2.3', '4.2.4', '4.2.5', '4.2.6', '4.2.7', '4.2.8', '4.2.9', '4.3.2', '4.3.3', '4.4.2.0', '4.4.2.1', '444.6.4.2.999']],
                    'SemVer3': [['3-oas3', '3.0', '3.0.0', '3.0.0-beta', '3.0.0-beta2', '3.0.0-draft', '3.0.0-rc.1', '3.0.0_RC', '3.0.1', '3.0.10', '3.0.11', '3.0.12', '3.0.13', '3.0.14', '3.0.2', '3.0.3', '3.0.4', '3.0.5', '3.0.6', '3.0.7', '3.0.8', '3.0.9', '3.0.x', '3.00', '3.1', '3.1.0', '3.1.0-rc.1', '3.1.1', '3.1.2', '3.1.3', '3.1.4', '3.1.5', '3.1.6', '3.1.7', '3.1.8', '3.10', '3.10.0', '3.11.0', '3.12.0', '3.12.1', '3.12.2', '3.13', '3.13.0', '3.14', '3.14.0', '3.15', '3.15.0', '3.16', '3.2.0', '3.2.1', '3.2.2', '3.3', '3.3.0', '3.4.0', '3.4.1', '3.5', '3.5.0', '3.6.0', '3.7', '3.7.0', '3.8', '3.8.0', '3.8.1', '3.9', '3.9.0', '3.9.1', '3.x', '30']],
                    'SemVer2': [['2.0', '2.0.0', '2.4.1', '2.4.1-rc1', '2.4.10', '2.4.2', '2.4.3','2.4.4', '2.5', '2.5.0', '2.5.1', '2.5.2', '2.6.0','2.6.1', '2.6.1-beta1', '2.6.2', '2.6.2-rc1', '2.6.5', '2.6.6', '2.6.7', '2.7.0', '2.7.1', '2.7.2', '2.8', '2.8.0', '2.8.1', '2.8.2', '2.8.3', '2.8.4', '2.8.5', '2.8.6', '2.9.0', '2.9.1', '2.9.2', '2.9.3', '2.9.4', '2.9.5', '2.9.7']],
                    'SingleVerNumber':[[ '1', '2', '3', '4', '5', '6', '7', '8']],
                    'SemVer0': [['0.1.0','0.3.4', '0.3.3', '0.3.2', '0.3.1', '0.3.0', '0.2.0', '0.3.5', '0.9.0', '0.0.1 (develop)', '0.2.3', '0.2.2', '0.2.1', '0.1.5', '0.1.4', '0.1.3', '0.1.2', '0.1.1', '0.0.1','0.103.0', '0.101.0', '0.91.0', '0.83.0', '0.77.0', '0.74.0', '0.73.0', '0.69.0', '0.64.0', '0.60.0', '0.59.0', '0.58.0', '0.55.0', '0.54.0', '0.51.0', '0.50.0', '0.49.0', '0.109.0', '0.115.0', '0.120.0', '0.123.0', '0.126.0','0.2.16', '0.2.8', '0.2.7', '0.2.4', '0.2.9','0.0.5','0.9.3', '0.9.2', '0.9.1', '0.8.20', '0.8.19', '0.8.18', '0.8.17', '0.8.16', '0.8.15', '0.8.14', '0.8.13', '0.8.12', '0.8.11', '0.8.10', '0.8.9', '0.8.8', '0.8.7', '0.8.6', '0.8.5', '0.8.4', '0.8.2', '0.8.1', '0.8.0', '0.7.2', '0.7.1', '0.7.0', '0.6.3', '0.6.2', '0.6.1', '0.6.0', '0.5.16', '0.5.15', '0.5.14', '0.5.13', '0.5.12', '0.5.11', '0.5.10', '0.5.9', '0.5.8', '0.5.7', '0.5.6', '0.5.5', '0.5.4', '0.10.5','0.17.8', '0.17.7', '0.17.6', '0.17.5', '0.17.4', '0.17.3', '0.17.0', '0.16.0', '0.15.2', '0.15.1', '0.15.0', '0.14.14', '0.14.8', '0.14.7', '0.14.6', '0.14.3', '0.14.2', '0.14.1', '0.14', '0.13.1', '0.13', '0.12.2', '0.12.1', '0.12', '0.9', '0.10', '0.8', '0.6', '0.3', '0.2','0.0.145', '0.0.143', '0.28.0','0.20190213.0', '0.20190115.0', '0.20190105.0', '0.20181229.0', '0.20181217.0', '0.20181211.0', '0.20181127.0', '0.20181117.0', '0.20181028.0', '0.20180905.0', '0.20180817.0', '0.20180815.0', '0.20180607.0', '0.20180519.0', '0.20180429.0', '0.20170106.0', '0.0.20170106', '0.0.20161219','0.4.10', '0.4.9', '0.4.8', '0.4.7', '0.4.6', '0.4.5', '0.4.4', '0.4.3', '0.4.2', '0.4.13', '0.4.12', '0.4.11', '0.5.3', '0.5.2', '0.5.1', '0.4.15', '0.4.14', '0.6.13', '0.6.12', '0.6.11', '0.6.10', '0.6.9', '0.6.8', '0.6.7', '0.6.6', '0.6.5', '0.6.4', '0.7.3', '0.7.7', '0.7.6', '0.7.5', '0.7.4', '0.7.8', '0.7.9', '0.7.11', '0.7.10', '0.7.12', '0.7.13', '0.8.3', '0.7.16', '0.7.15', '0.7.14','0.0','0.1.9','0.1.8', '0.0.6', '0.0.4', '0.0.7', '0.0.15', '0.0.16', '0.0.18', '0.0.17', '0.0.19', '0.0.20', '0.0.21', '0.0.27', '0.0.26', '0.0.25', '0.0.24', '0.0.23', '0.0.22','0.1.26', '0.1.25', '0.23.0','0.12.0', '0.13.0','0.18.0','0.14.0','0.11.0','0.10.0','0.32.0','0.33.0',  '0.35.0','0.34.0', '0.0.12', '0.0.11', '0.0.10', '0.0.9', '0.0.8', '0.1.234', '0.1.233', '0.1.232', '0.1.231', '0.1.230', '0.1.229', '0.1.228', '0.1.227', '0.1.226', '0.1.225', '0.1.224', '0.1.223', '0.1.222', '0.1.221', '0.1.220', '0.1.219', '0.1.218', '0.1.215', '0.1.214', '0.1.213', '0.1.212', '0.1.210', '0.1.209', '0.1.208', '0.1.207', '0.1.206', '0.1.205', '0.1.204', '0.1.203', '0.1.201', '0.1.200', '0.1.199', '0.1.198', '0.1.197', '0.1.196', '0.1.195', '0.1.194', '0.1.193', '0.1.192', '0.1.191', '0.1.190', '0.1.189', '0.1.188', '0.1.187', '0.1.186', '0.1.185', '0.1.183', '0.1.182', '0.1.180', '0.1.179', '0.1.178', '0.1.177', '0.1.176', '0.1.175', '0.1.174', '0.1.172', '0.1.170', '0.1.168', '0.1.165', '0.1.163', '0.1.161', '0.1.160', '0.1.159', '0.1.158', '0.1.157', '0.1.151', '0.1.150', '0.1.149', '0.1.148', '0.1.147', '0.1.146', '0.1.145', '0.1.144', '0.1.143', '0.1.141', '0.1.140', '0.1.139', '0.1.138', '0.1.105', '0.1.117', '0.1.116', '0.1.115', '0.1.114', '0.1.111', '0.1.104', '0.1.96', '0.1.95', '0.1.93', '0.1.92', '0.1.91', '0.1.90', '0.1.89', '0.1.88', '0.1.87', '0.1.86', '0.1.85', '0.1.84', '0.1.260', '0.1.259', '0.1.254', '0.1.253', '0.1.249', '0.1.248', '0.1.247', '0.1.246', '0.1.245', '0.1.244', '0.1.242', '0.1.241', '0.1.240', '0.1.239', '0.1.238', '0.1.237', '0.1.235', '0.1.284', '0.1.280', '0.1.279', '0.1.278', '0.1.277', '0.1.275', '0.1.274', '0.1.273', '0.1.272', '0.1.271', '0.1.269', '0.1.268', '0.1.267', '0.1.266', '0.1.265', '0.1.264', '0.1.263', '0.1.261', '0.1.290', '0.1.289', '0.1.288', '0.1.294', '0.1.293', '0.1.295', '0.1.304', '0.1.303', '0.1.302', '0.1.300', '0.1.299', '0.1.298', '0.1.297', '0.1.296', '0.1.324', '0.1.323', '0.1.317', '0.1.314', '0.1.311', '0.1.310', '0.1.309', '0.1.307', '0.1.328', '0.1.327', '0.1.326', '0.1.325', '0.1.348', '0.1.347', '0.1.346', '0.1.345', '0.1.343', '0.1.340', '0.1.333', '0.1.349', '0.1.357', '0.1.356', '0.1.355', '0.1.351', '0.1.350', '0.1.360', '0.1.359', '0.1.365', '0.1.363', '0.1.362', '0.1.361', '0.1.366', '0.1.368', '0.1.367', '0.1.373', '0.1.372', '0.1.371', '0.1.370', '0.1.369', '0.1.377', '0.1.376', '0.1.375', '0.1.374', '0.1.387', '0.1.386', '0.1.385', '0.1.384', '0.1.380', '0.1.379', '0.1.389', '0.1.388', '0.1.390', '0.1.392', '0.1.391','0.10.1', '0.11.3', '2.1', '0.3.43', '0.3.42', '0.3.41', '0.3.40', '0.3.39', '0.3.38', '0.3.37', '0.3.32', '0.3.31', '0.3.30', '0.3.29', '0.3.28', '0.3.27', '0.3.26', '0.3.25', '0.3.24', '0.3.23', '0.3.22', '0.3.21', '0.3.20', '0.3.19', '0.3.18', '0.3.17', '0.3.16', '0.3.15', '0.3.14', '0.3.13', '0.3.12', '0.3.11', '0.3.10', '0.3.9', '0.3.8', '0.3.7', '0.2.28', '0.2.27', '0.2.25', '0.2.24', '0.2.23', '0.3.44', '0.3.46', '0.3.45','0.8.3-dev', '0.8.2-dev', '0.7', '0.9.0-dev', '0.95.0', '0.94.0', '0.93.0', '0.92.0', '0.9.40', '0.9.39', '0.9.38', '0.9.37', '0.9.36', '0.9.35', '0.9.34', '0.9.33', '0.9.32', '0.9.31', '0.9.30', '0.9.29', '0.9.28', '0.9.27', '0.9.26', '0.9.52', '0.9.51', '0.9.50', '0.9.46', '0.9.45', '0.9.44', '0.9.43', '0.9.42', '0.9.41', '0.9.54', '0.9.53', '0.9.56', '0.9.55', '0.9.57', '0.9.58', '0.9.59', '0.9.60', '0.9.61', '0.9.63', '0.9.62', '0.9.64', '0.9.65', '0.9.68', '0.9.67', '0.9.66', '0.9.69','0.49.2', '0.49.1', '0.48.0', '0.47.2', '0.47.1', '0.47.0', '0.50.2', '0.50.1','0.1.15', '0.1.14', '0.1.13', '0.1.11', '0.1.10', '0.2.5','0.1.29', '0.1.28', '0.1.27', '0.1.24', '0.1.23', '0.1.22', '0.1.21', '0.1.20', '0.1.19', '0.1.18', '0.1.17', '0.1.16', '0.9.6', '0.9.5', '0.9.4', '0.9.8', '0.9.7','0.0.1317', '0.0.1315', '0.0.1313', '0.0.1311', '0.0.1309', '0.0.1307', '0.0.1305', '0.0.1303', '0.0.1301', '0.0.1299', '0.0.1293', '0.0.1291', '0.0.1289', '0.0.1287', '0.0.1285', '0.0.1283', '0.0.1281', '0.0.1279', '0.0.1277', '0.0.1273', '0.0.1271', '0.0.1262', '0.0.1260', '0.0.1254', '0.0.1252', '0.0.1250', '0.0.1248', '0.0.1246', '0.0.1244', '0.0.1242', '0.0.1240', '0.0.1238', '0.0.1236', '0.0.1234', '0.0.1232', '0.0.1230', '0.0.1226', '0.0.1224', '0.0.1222', '0.0.1220', '0.0.1214', '0.0.1190', '0.0.1188', '0.0.1186', '0.0.1184', '0.0.1176', '0.0.1172', '0.0.1170', '0.0.1369', '0.0.1367', '0.0.1365', '0.0.1363', '0.0.1361', '0.0.1359', '0.0.1357', '0.0.1355', '0.0.1353', '0.0.1351', '0.0.1349', '0.0.1347', '0.0.1345', '0.0.1343', '0.0.1341', '0.0.1339', '0.0.1337', '0.0.1335', '0.0.1333', '0.0.1331', '0.0.1329', '0.0.1319', '0.0.1435', '0.0.1433', '0.0.1431', '0.0.1424', '0.0.1422', '0.0.1418', '0.0.1413', '0.0.1403', '0.0.1395', '0.0.1393', '0.0.1391', '0.0.1389', '0.0.1387', '0.0.1385', '0.0.1383', '0.0.1381', '0.0.1379', '0.0.1375', '0.0.1373', '0.0.1493', '0.0.1491', '0.0.1489', '0.0.1487', '0.0.1485', '0.0.1483', '0.0.1481', '0.0.1479', '0.0.1477', '0.0.1475', '0.0.1473', '0.0.1471', '0.0.1469', '0.0.1467', '0.0.1465', '0.0.1461', '0.0.1455', '0.0.1453', '0.0.1451', '0.0.1449', '0.0.1447', '0.0.1445', '0.0.1443', '0.0.1441', '0.0.1439', '0.0.1437', '0.0.1519', '0.0.1517', '0.0.1515', '0.0.1513', '0.0.1509', '0.0.1507', '0.0.1505', '0.0.1503', '0.0.1501', '0.0.1499', '0.0.1495', '0.0.1521', '0.0.1575', '0.0.1569', '0.0.1567', '0.0.1563', '0.0.1561', '0.0.1559', '0.0.1557', '0.0.1555', '0.0.1553', '0.0.1551', '0.0.1549', '0.0.1547', '0.0.1543', '0.0.1541', '0.0.1539', '0.0.1537', '0.0.1535', '0.0.1533', '0.0.1531', '0.0.1529', '0.0.1527', '0.0.1523', '0.0.1589', '0.0.1587', '0.0.1583', '0.0.1579', '0.0.1602', '0.0.1600', '0.0.1598', '0.0.1591', '0.0.1610', '0.0.1608', '0.0.1606', '0.0.1624', '0.0.1622', '0.0.1620', '0.0.1618', '0.0.1616', '0.0.1614', '0.0.1632', '0.0.1626', '0.0.1809', '0.0.1807', '0.0.1805', '0.0.1803', '0.0.1801', '0.0.1799', '0.0.1797', '0.0.1795', '0.0.1793', '0.0.1791', '0.0.1789', '0.0.1787', '0.0.1785', '0.0.1783', '0.0.1781', '0.0.1779', '0.0.1777', '0.0.1775', '0.0.1773', '0.0.1771', '0.0.1769', '0.0.1765', '0.0.1763', '0.0.1761', '0.0.1759', '0.0.1757', '0.0.1753', '0.0.1751', '0.0.1747', '0.0.1745', '0.0.1743', '0.0.1741', '0.0.1739', '0.0.1737', '0.0.1728', '0.0.1726', '0.0.1724', '0.0.1720', '0.0.1718', '0.0.1716', '0.0.1714', '0.0.1712', '0.0.1710', '0.0.1706', '0.0.1702', '0.0.1698', '0.0.1694', '0.0.1686', '0.0.1684', '0.0.1682', '0.0.1680', '0.0.1678', '0.0.1676', '0.0.1674', '0.0.1672', '0.0.1670', '0.0.1668', '0.0.1664', '0.0.1662', '0.0.1656', '0.0.1654', '0.0.1652', '0.0.1648', '0.0.1646', '0.0.1640', '0.0.1636', '0.0.1634', '0.0.1833', '0.0.1823', '0.0.1821', '0.0.1819', '0.0.1817', '0.0.1815', '0.0.1811', '0.0.1853', '0.0.1849', '0.0.1847', '0.0.1845', '0.0.1843', '0.0.1841', '0.0.1839', '0.0.1837', '0.0.1835', '0.0.1857', '0.0.1855','0.9.23', '0.9.22', '0.9.21', '0.9.20', '0.9.19', '0.9.18', '0.9.17', '0.9.16', '0.9.15', '0.9.14', '0.9.13', '0.9.12', '0.9.11', '0.9.10', '0.9.9','0.2.26','0.83.1','0.1.258', '0.1.181', '0.1.142', '0.1.83', '0.1.82', '0.1.81', '0.1.80', '0.1.79', '0.1.78', '0.1.73', '0.18.1','0.1.13-2', '0.0.31', '0.0.30','0.90.3', '0.90.2', '0.90.1']],
                    'Snapshots': [['19.3.6-SNAPSHOT', '19.3.5-SNAPSHOT','19.6.1-SNAPSHOT','1.0.2-SNAPSHOT','1.0.0-SNAPSHOT','1.0.6-SNAPSHOT','1.0.5-SNAPSHOT','1.0.4-SNAPSHOT','1.10.0-alpha.0-SNAPSHOT', '1.9.0-beta.3-SNAPSHOT', '1.9.0-beta.1-SNAPSHOT''1.9.0-alpha.9-SNAPSHOT','1.9.0-alpha.6-SNAPSHOT', '1.9.0-alpha.5-SNAPSHOT', '1.9.0-alpha.4-SNAPSHOT','1.9.0-alpha.1-SNAPSHOT','1.1.0-SNAPSHOT','1.0-SNAPSHOT','0.0.1-SNAPSHOT','0.1-SNAPSHOT','0.6.0-SNAPSHOT', '0.4.0-SNAPSHOT','0.21.0-SNAPSHOT', '0.20.0-SNAPSHOT','0.19.0-SNAPSHOT', '0.18.0-SNAPSHOT', '0.17.0-SNAPSHOT', '0.16.0-SNAPSHOT', '0.15.0-SNAPSHOT','0.14.0-SNAPSHOT', '0.13.0-SNAPSHOT', '0.12.0-SNAPSHOT','0.11.0-SNAPSHOT','0.10.0-SNAPSHOT', '0.9.0-SNAPSHOT', '0.8.0-SNAPSHOT', '0.28.0-SNAPSHOT','0.27.0-SNAPSHOT', '21.12-SNAPSHOT','0.26.0-SNAPSHOT','0.31.0-SNAPSHOT', '0.30.0-SNAPSHOT','0.29.0-SNAPSHOT', '0.32.0-SNAPHSOT','0.32.0-SNAPSHOT', '0.33.0-SNAPSHOT','0.34.0-SNAPSHOT', '0.36.0-SNAPSHOT','0.35.0-SNAPSHOT','0.37.0-SNAPSHOT','0.38.0-SNAPSHOT','20.14.0-SNAPSHOT','0.0.8-SNAPSHOT', '2.3.0-SNAPSHOT', '2.5.1-SNAPSHOT', '2.6.0-SNAPSHOT', '2.4.3-SNAPSHOT', '2.4.2-SNAPSHOT', '2.2.2-SNAPSHOT', '2.2.1-SNAPSHOT', '2.5.4-SNAPSHOT', '2.5.3-SNAPSHOT', '2.5.2-SNAPSHOT', '2.5.0-SNAPSHOT', '2.6.2-SNAPSHOT', '2.6.1-SNAPSHOT','12.0.1-SNAPSHOT', '11.0.4-SNAPSHOT','11.0.3-SNAPSHOT', '-SNAPSHOT','19.0.8-SNAPSHOT','19.0.7-SNAPSHOT', '19.0.6-SNAPSHOT','19.0.9-SNAPSHOT','0.8.1-SNAPSHOT', '0.7.1-SNAPSHOT', '0.10.1-SNAPSHOT','3.3-SNAPSHOT','0.68.0-SNAPSHOT', '0.66.0-SNAPSHOT', '0.65.0-SNAPSHOT', '0.62.0-SNAPSHOT', '0.44.0-SNAPSHOT','20.0.1-SNAPSHOT','21.0.1-SNAPSHOT','22.0.1-SNAPSHOT','2.19-SNAPSHOT']],
                    'Timestamps':[['20201127-10:51:06.1ec1f7d', '20201028-17:09:36', '20201016-08:58:07','20201028-12:58:23', '20201015-11:05:51', '20200928-09:21:39', '20200922-14:31:35', '20200921-09:54:40', '20200917-08:15:54', '20200916-13:32:45.729020a', '20200910-11:46:01.1fa2fb5', '20200909-09:42:23.56b2a71', '20200908-12:07:40.ed48938','2020-08-20T12:57:36Z','2018-06-28', '2018052300','20200717-15:16:13.5a90460','2016-11-19T00:21:31.413+0000','2019-03-04', '2019-02-21-16:00','2019-03-13','2020-04-04T07:24:59Z','2018-11-09T16:17:31Z', '2018-11-02T15:17:31Z']],
                    'Numbered Versions': [['v0','v1', 'v2', 'v3', 'v4', 'v01','V1.0','v.04', 'v.03', 'v.02', 'v.01','v.9.7', 'v.9.6', 'v.9.4', 'v.9.2', 'v.9.0', 'v.8.1', 'v.7.0.1', 'v.7.0', 'v.6.0', 'v.5.4.4', 'v.5.4.2', 'v.4.3', 'v.4.2', 'v.4.1', 'v.2.0', 'v.1.2']],
                    'Version_with_dev_stage': [['v0.6.0-alpha.6', 'v0.6.0-alpha.1', 'v0.0.0-alpha.58', 'v0.5.7-alpha.1.pre.0', 'v0.0.0-alpha.46', 'v0.0.0-alpha.45', 'v0.0.0-alpha.44', 'v0.0.0-alpha.43', 'v0.0.0-alpha.42', 'v0.0.0-alpha.41', 'v0.0.0-alpha.40', 'v0.7.0-alpha.1', 'v0.7.0-alpha.0', 'v0.8.0-alpha.1', 'v0.8.0-alpha.2', 'v0.9.0-alpha.0','v1alpha', 'v1alpha1','v2beta1.4', 'v2beta1.3', 'v2beta1.1','v1.0.98-alpha', 'Dev - Eth2Spec v1.0.1', 'V1 - Eth2Spec v1.0.1', 'Dev - Eth2Spec v0.12.2', 'Dev - Eth2Spec v0.11.1','v1beta2.32', 'v1beta2.17', 'v1beta2.16', 'v1beta2.14', 'v1beta2.12', 'v1beta2.9']],
                    'Year of Release':[['210720', '210628', '210521', '210511', '210427', '210406', '210315', '210208', '210129', '210125', '210111', '201217', '201207', '201026', '201014', '201005', '200928', '200914', '200901', '200817', '200805', '200730', '200713', '200626', '200615', '200602', '200518', '200505', '200421', '200402', '200330', '200310', '200303', '200218', '200203', '200121', '191029', '191017', '191015', '191001', '190916', '190903', '190828', '190820', '190717', '190627', '190624', '190605', '190520', '190509', '190424', '190410', '211209', '211208', '211108', '211025', '211012', '210927', '210913', '210830', '210816', '210802', '220128', '220118', '220214', '220301', '220305', '220621', '220607', '220523', '220425', '220411', '220406', '220328', '220321', '220316', '220802', '220801', '220718', '220704', '220829', '220815', '220907', '200629', '200311', '191218', '191121', '191014', '190716', '190610', '190312', '190301', '190225', '210911', '220606', '220630', '220908', '2019.0.0']],
                    'SemVer17': [['17.2.8', '17.2.6', '17.2.5', '17.2.4', '17.2.3', '17.2.2', '17.1.3']],
                    'SemVer18': [['18.2.8', '18.2.7', '18.2.6', '18.2.5', '18.2.4', '18.2.3', '18.2.2', '18.1.3', '18.1.2']],
                    'SemVer19':[['19.7.1', '19.7.0', '19.6.2', '19.6.1', '19.6.0', '19.5.0', '19.3.8', '19.3.7', '19.3.4']],
                    'SemVer10': [['10.1.0', '10.0.1', '10.0.0']],
                    'SemVer20': [['20.1.3', '20.1.2', '20.1.1']],
                    'SemVer21': [['22.1.1', '22.1.3']],
                    'Numbers': [['30', '25', '18','1', '2', '3', '4', '5','6','0', '7', '8', '9','10']],
                    'SemVer8': [['8.10.1', '8.10.0', '8.9.0', '8.8.1', '8.8.0', '8.7.1', '8.7.0', '8.6.0', '8.5.0', '8.4.1', '8.4.0', '8.3.2', '8.3.1', '8.3.0', '8.2.0', '8.1.0', '8.0.0', '5.12.2', '5.12.1', '8.11.0', '8.10.2', '8.12.1', '8.11.1', '8.17.0', '8.16.0', '8.15.0', '8.14.0', '8.13.0', '8.24.0', '8.23.0', '8.22.0', '8.21.0', '8.20.0', '8.19.3', '8.19.2', '8.19.1', '8.19.0', '8.18.0', '8.27.2', '8.27.1', '8.27.0', '8.26.1', '8.26.0', '8.25.0', '8.24.1', '8.28.2', '8.28.1', '8.29.3', '8.29.2', '8.29.1', '8.29.0', '8.29.5', '8.29.4', '8.30.1', '8.30.0', '8.32.0', '8.31.3', '8.31.2', '8.31.1', '8.31.0', '8.32.1', '8.32.4', '8.32.3', '8.32.2', '8.32.6', '8.32.5', '8.37.1', '8.37.0', '8.36.0', '8.35.1', '8.35.0', '8.34.0', '8.39.0', '8.38.0', '8.41.0', '8.40.1', '8.40.0', '8.46.0', '8.44.0', '8.43.0', '8.42.0', '8.41.1', '8.54.0', '8.52.1', '8.52.0', '8.51.0', '8.50.0', '8.49.0', '8.48.0', '8.47.0', '8.57.0', '8.56.0', '8.55.0']],
                    'Sem Ver with dev status': [['0.9.44 (BETA)', '0.9.43 (BETA)', '0.9.42 (BETA)', '0.9.41 (BETA)', '0.9.40 (BETA)', '0.9.39 (BETA)', '0.9.38 (BETA)', '0.9.37 (BETA)', '0.9.36 (BETA)', '0.9.35 (BETA)', '0.9.34 (BETA)', '0.9.33 (BETA)', '0.9.32 (BETA)', '0.9.31 (BETA)', '0.9.30 (BETA)', '0.9.29 (BETA)', '0.9.28 (BETA)', '0.9.27 (BETA)', '0.9.26 (BETA)', '0.9.52 (BETA)', '0.9.51 (BETA)', '0.9.50 (BETA)', '0.9.46 (BETA)', '0.9.45 (BETA)', '0.9.54 (BETA)', '0.9.53 (BETA)', '0.9.55 (BETA)', '0.9.56 (BETA)', '0.9.57 (BETA)', '0.9.59 (BETA)', '0.9.58 (BETA)', '0.9.60 (BETA)', '0.9.61 (BETA)', '0.9.62 (BETA)', '0.9.63 (BETA)', '0.9.64 (BETA)', '0.9.65 (BETA)', '0.9.66 (BETA)', '0.9.68 (BETA)', '0.9.67 (BETA)', '0.9.69 (BETA)']],
                    'Months':[['May 2021', 'Dec 2021', 'Sep 2021', 'Jan 2022','2021.02']],
                    'SemVer1': [['1.1.0','1.0.2','1.0.1','1.0.0','1.0.3','1.3.0', '1.2.0','1.0.5','1.0.4','1.0.6', '1.0.11', '1.0.10', '1.0.9', '1.0.8', '1.0.7', '1.0.14', '1.0.15','1.3.1', '1.5.0', '1.4.0', '1.6.0','1.7.6', '1.10.0', '1.11.0','1.0.36', '1.0.35', '1.0.34', '1.0.33', '1.0.32', '1.0.31', '1.0.30', '1.0.29', '1.0.28', '1.0.27', '1.0.26', '1.0.25', '1.0.24', '1.0.23', '1.0.22', '1.0.21', '1.0.20', '1.0.19', '1.0.18', '1.0.17', '1.0.16', '1.0.13', '1.0.12','1.2.2', '1.2.1', '1.5.1', '1.2.3', '1.6.1', '1.7.0', '1.7.2', '1.7.1', '1.7.3', '1.0.1.1', '1.1.3', '1.1.2', '1.3.2', '1.3.3','1.1.1','1.19.0', '1.18.0', '1.17.0', '1.16.1', '1.16.0', '1.15.0', '1.14.0', '1.13.0', '1.12.0', '1.9.0', '1.8.0', '1.19.1', '1.20.3', '1.20.2', '1.20.1', '1.20.0', '1.24.0', '1.23.2', '1.23.1', '1.23.0', '1.22.0', '1.21.0', '1.25.0', '1.25.1', '1.27.1', '1.27.0', '1.26.0', '1.27.2', '1.28.0', '1.28.1', '1.28.2', '1.29.0', '1.29.1', '1.29.2', '1.34.0', '1.33.0', '1.32.0', '1.31.0', '1.30.0','1.0.0.1','1.4.4', '1.4.6', '1.4.5', '1.4.8', '1.4.7', '1.4.9', '1.4.10', '1.3.4', '1.3.5','1.1.7', '1.1.4','1.5.3', '1.5.2', '1.7.5', '1.7.4','1.1.5', '1.1.7.4', '1.1.7.6', '1.1.7.8', '1.4.2', '1.4.1', '1.6.3', '1.6.2', '1.3.6', '1.2.6', '1.2.5', '1.2.4', '1.8.1', '1.8.4','1.10', '1.12', '1.11', '1.17', '1.16', '1.15', '1.14', '1.13', '1.18','1.2.25', '1.2.27', '1.2.26','1.6.5', '1.6.4','1.62.0', '1.61.1', '1.60.0', '1.59.1', '1.58.0', '1.57.0', '1.56.0', '1.55.4', '1.55.1', '1.54.2', '1.53.13', '1.52.0', '1.51.0', '1.50.0', '1.49.0', '1.48.1', '1.48.0','1.47.0', '1.46.3', '1.46.0','1.1','1.9.3', '1.9.2', '1.9.4','1.2.8', '1.2.7', '1.3.7', '1.3.8','1.13.3', '1.13.2', '1.13.1', '1.12.2', '1.11.4', '1.11.3', '1.11.2', '1.11.1', '1.10.2', '1.10.1', '1.13.4','1.0.0','1.3.6.0', '1.3.4.0', '1.3.3.0', '1.3.2.0', '1.3.8.0', '1.3.9.0','1.21.2', '1.21.1', '1.17.3', '1.17.2', '1.17.1', '1.15.1', '1.21.4', '1.21.2', '1.21.1', '1.17.3', '1.17.2', '1.17.1', '1.15.1', '1.21.4','1.21.3','1.245.259', '1.239.207', '1.237.854', '1.236.925', '1.236.237', '1.236.100', '1.235.456', '1.235.67', '1.234.135', '1.233.836', '1.232.896', '1.231.909', '1.230.178', '1.230.0', '1.229.796', '1.228.877', '1.228.582', '1.228.121', '1.228.97', '1.227.883', '1.227.290', '1.226.959', '1.226.680', '1.226.376', '1.226.157', '1.226.49', '1.225.712', '1.225.541', '1.225.497', '1.225.430', '1.223.826', '1.223.771', '1.223.605', '1.223.70', '1.222.778', '1.222.632', '1.222.104', '1.222.89', '1.221.905', '1.221.843', '1.220.899', '1.220.764', '1.220.91', '1.219.637', '1.219.457', '1.219.343', '1.218.904', '1.218.331', '1.218.23', '1.217.753', '1.216.924', '1.216.910', '1.216.840', '1.216.736', '1.216.706', '1.216.628', '1.216.473', '1.216.178', '1.216.52', '1.215.236', '1.213.501', '1.213.356', '1.212.374', '1.212.90', '1.211.793', '1.211.502', '1.210.639', '1.210.535', '1.209.645', '1.207.977', '1.207.390', '1.207.58', '1.206.755', '1.206.425', '1.206.232', '1.205.973', '1.205.266', '1.205.228', '1.205.64', '1.205.22', '1.204.545', '1.204.226', '1.203.920', '1.203.897', '1.203.800', '1.203.697', '1.203.627', '1.203.284', '1.202.863', '1.202.485', '1.202.410', '1.202.232', '1.202.64', '1.201.857', '1.201.722', '1.201.332', '1.200.588', '1.200.411', '1.199.160', '1.198.827', '1.198.629', '1.198.139', '1.198.94', '1.197.33', '1.196.832', '1.196.634', '1.196.424', '1.195.934', '1.195.444', '1.195.277', '1.195.206', '1.195.51', '1.195.37', '1.194.952', '1.194.798', '1.194.462', '1.194.438', '1.194.365', '1.194.314', '1.194.195', '1.194.180', '1.194.126', '1.194.9', '1.193.847', '1.193.647', '1.193.514', '1.193.343', '1.192.600', '1.192.577', '1.192.271', '1.191.644', '1.191.312', '1.190.862', '1.255.419', '1.255.171', '1.254.777', '1.254.686', '1.254.439', '1.254.277', '1.253.757', '1.253.382', '1.253.120', '1.252.735', '1.273.170', '1.272.573', '1.272.43', '1.271.782', '1.271.471', '1.270.370', '1.269.473', '1.268.704', '1.267.718', '1.266.433', '1.266.175', '1.264.173', '1.263.145', '1.258.249', '1.256.472', '1.256.271', '1.256.139', '1.255.903', '1.255.850', '1.255.592', '1.277.863', '1.275.824', '1.283.18', '1.282.255', '1.280.385', '1.293.147', '1.293.49', '1.300.693', '1.300.88', '1.302.488', '1.303.587', '1.307.810', '1.315.269', '1.314.864', '1.313.493', '1.312.660', '1.317.660', '1.317.812','1.52.1', '1.37', '1.36', '1.35', '1.34', '1.33', '1.32', '1.31', '1.30', '1.29', '1.28', '1.27', '1.26', '1.25', '1.0.100','1.40', '1.39', '1.38', '1.5','1.1.12', '1.1.11', '1.1.10', '1.1.9','1.0.31.11012', '1.0.30.9336', '1.0.29.61544', '1.0.28.47824', '1.0.27.32591', '1.0.26.29322', '1.0.25.28005', '1.0.24.43799', '1.0.22.24362', '1.0.20.17911', '1.0.19.61352', '1.0.18.40231', '1.0.9.31997', '1.0.8.7469', '1.0.7.12475', '1.0.6.46078', '1.0.5.51985', '1.0.4.52481', '1.0.48.55357', '1.0.47.9601', '1.0.46.35170', '1.0.45.47419', '1.0.44.14346', '1.0.43.35053', '1.0.42.1492', '1.0.41.9687', '1.0.40.59515', '1.0.39.8844', '1.0.38.45384', '1.0.37.220', '1.0.36.5532', '1.0.35.42083', '1.0.34.2433', '1.0.33.58700', '1.0.32.17501', '1.0.52.44920', '1.0.51.3483', '1.0.49.26149', '1.0.70.11062', '1.0.69.60282', '1.0.68.63678', '1.0.67.16272', '1.0.66.48826', '1.0.65.10841', '1.0.64.9849', '1.0.63.51822', '1.0.62.663', '1.0.61.13002', '1.0.60.8667', '1.0.59.65388', '1.0.58.62354', '1.0.57.26586', '1.0.56.33990', '1.0.55.29177', '1.0.54.48930', '1.0.53.23746', '1.0.86.53123', '1.0.85.47026', '1.0.84.36506', '1.0.83.52475', '1.0.82.24145', '1.0.81.2593', '1.0.80.24777', '1.0.79.39248', '1.0.78.14081', '1.0.77.40848', '1.0.76.45870', '1.0.75.39172', '1.0.74.716', '1.0.73.2478', '1.0.72.52227', '1.0.71.46477', '1.0.102.51682', '1.0.101.41271', '1.0.100.35171', '1.0.99.10562', '1.0.98.57042', '1.0.97.15512', '1.0.96.7197', '1.0.95.15507', '1.0.94.26709', '1.0.93.52130', '1.0.91.48247', '1.0.90.33014', '1.0.88.21159', '1.0.87.4865', '1.0.106.25115', '1.0.105.54470', '1.0.104.8018', '1.0.103.57782', '1.0.110.51888', '1.0.109.3849', '1.0.108.26277', '1.0.107.12764', '1.0.119.14308', '1.0.118.24152', '1.0.117.53657', '1.0.116.56504', '1.0.115.37279', '1.0.114.27670', '1.0.113.59152', '1.0.112.59954', '1.0.111.22676', '1.0.176.4898', '1.0.175.17400', '1.0.174.38358', '1.0.172.2177', '1.0.171.23326', '1.0.169.21531', '1.0.168.15972', '1.0.167.27327', '1.0.166.15714', '1.0.164.37989', '1.0.163.55297', '1.0.162.36596', '1.0.161.51748', '1.0.160.16950', '1.0.159.55158', '1.0.158.30160', '1.0.157.38007', '1.0.156.20927', '1.0.155.49148', '1.0.154.666', '1.0.153.8758', '1.0.152.52380', '1.0.151.6551', '1.0.150.13982', '1.0.149.24513', '1.0.190.52159', '1.0.189.48709', '1.0.188.36403', '1.0.187.61442', '1.0.186.10854', '1.0.185.5397', '1.0.184.54484', '1.0.183.2721', '1.0.182.16417', '1.0.181.4906', '1.0.180.54899', '1.0.179.44311', '1.0.178.45030', '1.0.177.26035', '1.0.197.40755', '1.0.196.34613', '1.0.195.63826', '1.0.194.12673', '1.0.193.65513', '1.0.192.31572', '1.0.191.30522', '1.0.205.44002', '1.0.204.22379', '1.0.203.131', '1.0.202.38027', '1.0.201.30912', '1.0.200.43344', '1.0.199.14157', '1.0.198.4797', '1.0.213.45223', '1.0.212.29830', '1.0.210.2124', '1.0.209.42225', '1.0.208.1878', '1.0.207.30089', '1.0.206.49506', '1.0.229.55090', '1.0.228.62144', '1.0.227.32477', '1.0.226.44685', '1.0.225.44351', '1.0.224.30340', '1.0.223.64988', '1.0.222.22713', '1.0.221.31033', '1.0.220.28217', '1.0.218.17442', '1.0.217.35741', '1.0.216.36664', '1.0.215.37543', '1.0.214.46647', '1.0.234.17069', '1.0.233.522', '1.0.231.42617', '1.0.230.30325', '1.0.255.55527', '1.0.254.41946', '1.0.253.36967', '1.0.252.38114', '1.0.251.29281', '1.0.250.22831', '1.0.249.4101', '1.0.248.12732', '1.0.247.43000', '1.0.246.64388', '1.0.244.14425', '1.0.243.10564', '1.0.242.29128', '1.0.241.29218', '1.0.240.59386', '1.0.239.22258', '1.0.238.45472', '1.0.237.21337', '1.0.236.51359', '1.0.235.33525', '1.0.278.23546', '1.0.277.46811', '1.0.276.35380', '1.0.275.51302', '1.0.274.53623', '1.0.273.36726', '1.0.272.8601', '1.0.271.12832', '1.0.270.48887', '1.0.269.45721', '1.0.268.60936', '1.0.267.41626', '1.0.266.43611', '1.0.265.56482', '1.0.264.55666', '1.0.262.48056', '1.0.261.52537', '1.0.260.58144', '1.0.259.9166', '1.0.258.41896', '1.0.257.2225', '1.0.308.1254', '1.0.307.761', '1.0.306.7694', '1.0.305.58912', '1.0.304.13587', '1.0.303.2499', '1.0.302.36850', '1.0.301.22203', '1.0.300.11467', '1.0.299.36749', '1.0.298.38620', '1.0.297.31694', '1.0.296.1890', '1.0.295.31381', '1.0.294.28193', '1.0.293.45527', '1.0.292.476', '1.0.291.19874', '1.0.290.54735', '1.0.289.2018', '1.0.288.49267', '1.0.287.63933', '1.0.286.35973', '1.0.285.28909', '1.0.284.54123', '1.0.283.47010', '1.0.282.33449', '1.0.280.19855', '1.0.279.52649', '1.0.316.40225', '1.0.315.47536', '1.0.314.54146', '1.0.313.58041', '1.0.312.34603', '1.0.311.7056', '1.0.310.22495', '1.0.309.13738', '1.0.337.59932', '1.0.336.58655', '1.0.335.20540', '1.0.334.57289', '1.0.333.47292', '1.0.332.6248', '1.0.331.64911', '1.0.329.54740', '1.0.328.48681', '1.0.327.3955', '1.0.326.17735', '1.0.325.4420', '1.0.324.7093', '1.0.323.58858', '1.0.322.53731', '1.0.321.8828', '1.0.320.31984', '1.0.318.2703', '1.0.317.23595','1.155.0', '1.153.0', '1.146.0', '1.145.1', '1.144.1', '1.126.10','1.2.15', '1.2.38', '1.2.37', '1.2.17' ]],
                    'Version with alphabets': [['1.0b4', '1.0b3', '1.0b1-subtaxa', '1.0b1', '1.0a19', '1.0a18', '1.0a17', '1.0a16', '1.0a15', '1.0a14', '1.0a13', '1.0a12', '1.0a11', '1.0a10', '1.0a9', '1.0a8', '1.0a7', '1.0a6', '1.0a5', '1.0a4', '1.0a3', '1.0a2', '1.0a1', '1.2.0dev','1.3.10 constant-champ', '1.3.9 cumulative-hardin', '1.3.8 indivisible-esau', '1.3.7 frequent-amara', '1.3.6 crooked-daija', '1.3.5 everlasting-ashanti', '1.3.4 breezy-leafy', '1.3.11 lucky-fremont', '1.3.13 bordering-jerilyn']

]})

arr = df1.columns

def func(row):
    name = [i for i in arr if np.isin(df1[i].values[0], row['info_version']).any()]
    if len(name) > 0:
        return name[0]
    else:
        return 'Undefined'


api_history.loc[:, 'Version']=api_history.apply(func, axis=1)

In [12]:
display(api_history)

,id,repo_name,owner,filename,commits,commitDate,api_spec_id,api_version,api_title,processed_at,openapi,info_description,info_version,info_title,externalDocs,servers_description,servers_urls,Version
0,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1617799976000,84,1.1.0,SQAaaS API,1639562016000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
1,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1622430757000,84,1.1.0,SQAaaS API,1639562021000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
2,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1624258233000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
3,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1623993083000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
4,124,challenge-schemas,Sage-Bionetworks,openapi.yaml,37,1607286945000,124,0.1.0,Registry of Open Community Challenge API,1639561989000,3.0.2,The OpenAPI specification implemented by the C...,0.1.0,Registry of Open Community Challenge API,,[],[{protocol}://rocc.com/{environment}/v1],SemVer0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67225,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1578550863000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.8,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer18
67226,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1574229808000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.7,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer18
67227,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1573786228000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,20.1.1,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer20
67228,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1569918516000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.6,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer18


In [22]:
api_history['Version'].value_counts()

SemVer1                    25407
Undefined                  15262
SemVer0                    11966
Numbered Versions           2918
SemVer18                    2584
SemVer2                     1630
SemVer3                     1473
SemVer17                    1393
SemVer20                    1249
SingleVerNumber              731
SemVer21                     513
Snapshots                    318
SemVer5                      298
Year of Release              243
Names                        210
Version_with_dev_stage       208
SemVer4                      176
SemVer7                      154
Sem Ver with dev status      146
SemVer8                      100
Numbers                       64
SemVer6                       51
Timestamps                    45
Version with alphabets        44
Months                        31
SemVer19                      11
SemVer10                       5
Name: Version, dtype: int64

In [ ]:
api_history["Version"] = api_history["Version"].astype("category") # convert the version column to a categorical variable

In [13]:
pip install plotly # install plotly for interactive graph plots

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


In [15]:
api_history[api_history['servers_urls'].map(lambda d: len(d)) > 0] # Removes empty values from servers_urls column

,id,repo_name,owner,filename,commits,commitDate,api_spec_id,api_version,api_title,processed_at,openapi,info_description,info_version,info_title,externalDocs,servers_description,servers_urls,Version
0,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1617799976000,84,1.1.0,SQAaaS API,1639562016000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
1,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1622430757000,84,1.1.0,SQAaaS API,1639562021000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
2,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1624258233000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
3,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1623993083000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1
4,124,challenge-schemas,Sage-Bionetworks,openapi.yaml,37,1607286945000,124,0.1.0,Registry of Open Community Challenge API,1639561989000,3.0.2,The OpenAPI specification implemented by the C...,0.1.0,Registry of Open Community Challenge API,,[],[{protocol}://rocc.com/{environment}/v1],SemVer0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67140,224433,gomoney-mock-epl,barisere,openapi.yaml,11,1613422578000,224433,0.0.1,Mock EPL Server,1638983117000,3.0.3,This is the API server documentation for our m...,0.0.1,Mock EPL Server,Find out more about this API,[Local development server],"[, http://localhost:{port}]",SemVer0
67141,224433,gomoney-mock-epl,barisere,openapi.yaml,11,1613407199000,224433,0.0.1,Mock EPL Server,1638983117000,3.0.3,This is the API server documentation for our m...,0.0.1,Mock EPL Server,Find out more about this API,[Local development server],"[, http://localhost:{port}]",SemVer0
67142,224433,gomoney-mock-epl,barisere,openapi.yaml,11,1613291477000,224433,0.0.1,Mock EPL Server,1638983117000,3.0.3,This is the API server documentation for our m...,0.0.1,Mock EPL Server,Find out more about this API,[Local development server],"[, http://localhost:{port}]",SemVer0
67169,224530,LearnWords,rolling-scopes,api.yaml,28,1586383905000,224530,1.0.0,Learn Words,1638983132000,3.0.0,a backend part of RSSchool Stage#2 task,1.0.0,Learn Words,,[],[/],SemVer1


In [16]:
# Categorize into versions based on list of substrings found in the URL's of the server.
words = {
    "v0",
    "v1",
    "v2",
    "v3",
    "v1.0",
    "v3.1",
    "v3.2",
    "0.1.0",
    "3.1",
    "v0.0.2",
    "api",
    "api/v1",
    "1.0.0",
}

def word_finder(urls):
    df_words = set(x for url in urls for x in url.split('/'))
    extract_words =  words.intersection(df_words)
    return ', '.join(extract_words)

api_history['server_version'] = api_history["servers_urls"].apply(word_finder)

In [17]:
# Replace empty values in server_version with NaN (to better the output of graphs)
import numpy as np
api_history['server_version'].replace('', np.nan, inplace=True)


In [18]:
api_history[['servers_urls', 'server_version']]

,servers_urls,server_version
0,"[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",v1
1,"[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",v1
2,"[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",v1
3,"[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",v1
4,[{protocol}://rocc.com/{environment}/v1],v1
...,...,...
67225,[],NaN
67226,[],NaN
67227,[],NaN
67228,[],NaN


In [19]:
# Adds a new column to the dataframe with the count of the Server version across the dataset
api_history['servercount'] = api_history.groupby('server_version')['server_version'].transform('count')

In [20]:
display(api_history)

,id,repo_name,owner,filename,commits,commitDate,api_spec_id,api_version,api_title,processed_at,openapi,info_description,info_version,info_title,externalDocs,servers_description,servers_urls,Version,server_version,servercount
0,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1617799976000,84,1.1.0,SQAaaS API,1639562016000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0
1,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1622430757000,84,1.1.0,SQAaaS API,1639562021000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0
2,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1624258233000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0
3,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,1623993083000,84,1.1.0,SQAaaS API,1639562027000,3.0.0,OpenAPI specification for the Software and Ser...,1.1.0,SQAaaS API,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0
4,124,challenge-schemas,Sage-Bionetworks,openapi.yaml,37,1607286945000,124,0.1.0,Registry of Open Community Challenge API,1639561989000,3.0.2,The OpenAPI specification implemented by the C...,0.1.0,Registry of Open Community Challenge API,,[],[{protocol}://rocc.com/{environment}/v1],SemVer0,v1,2090.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67225,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1578550863000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.8,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer18,NaN,NaN
67226,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1574229808000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.7,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer18,NaN,NaN
67227,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1573786228000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,20.1.1,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer20,NaN,NaN
67228,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,1569918516000,224665,None,None,,None,CLI\n```\n- configure testsedatastorelevel2 <k...,18.2.6,Avi TestSeDatastoreLevel2 Object API,,[],[],SemVer18,NaN,NaN


In [21]:
# Plot the Server Version categories and their counts
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
px.histogram(api_history, x="server_version",text_auto=True,width=1300, height = 900).update_xaxes(categoryorder="total descending", tickangle=40)

In [23]:
# Plot the  Version categories for API's and their counts
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
px.histogram( api_history, x="Version",text_auto=True,width=1700, height = 900,color = 'Version')

In [24]:
# Convert numberLong date format to timestamp
api_history['commitDate'] = pd.to_datetime(api_history['commitDate'], unit='ms')
api_history['processed_at'] = pd.to_datetime(api_history['processed_at'], unit='ms')


In [25]:
# Extract year and month from commitDate column
import calendar
api_history['Year'] = pd.DatetimeIndex(api_history['commitDate']).year
api_history['Month'] = pd.DatetimeIndex(api_history['commitDate']).month
api_history['Month'] = api_history['Month'].apply(lambda x: calendar.month_abbr[x])

In [26]:
#   Extract timestamp as seperate column
api_history['Time'] = pd.to_datetime(api_history['commitDate'])
api_history['timestamp'] = api_history['Time'].dt.time

In [27]:
display(api_history)

,id,repo_name,owner,filename,commits,commitDate,api_spec_id,api_version,api_title,processed_at,...,externalDocs,servers_description,servers_urls,Version,server_version,servercount,Year,Month,Time,timestamp
0,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,2021-04-07 12:52:56,84,1.1.0,SQAaaS API,2021-12-15 09:53:36,...,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0,2021,Apr,2021-04-07 12:52:56,12:52:56
1,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,2021-05-31 03:12:37,84,1.1.0,SQAaaS API,2021-12-15 09:53:41,...,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0,2021,May,2021-05-31 03:12:37,03:12:37
2,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,2021-06-21 06:50:33,84,1.1.0,SQAaaS API,2021-12-15 09:53:47,...,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0,2021,Jun,2021-06-21 06:50:33,06:50:33
3,84,sqaaas-api-spec,EOSC-synergy,openapi.yaml,32,2021-06-18 05:11:23,84,1.1.0,SQAaaS API,2021-12-15 09:53:47,...,,"[Main (production) API server, Staging API ser...","[http://api.sqaaas.eosc-synergy.eu/v1, https:/...",SemVer1,v1,2090.0,2021,Jun,2021-06-18 05:11:23,05:11:23
4,124,challenge-schemas,Sage-Bionetworks,openapi.yaml,37,2020-12-06 20:35:45,124,0.1.0,Registry of Open Community Challenge API,2021-12-15 09:53:09,...,,[],[{protocol}://rocc.com/{environment}/v1],SemVer0,v1,2090.0,2020,Dec,2020-12-06 20:35:45,20:35:45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67225,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,2020-01-09 06:21:03,224665,None,None,NaT,...,,[],[],SemVer18,NaN,NaN,2020,Jan,2020-01-09 06:21:03,06:21:03
67226,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,2019-11-20 06:03:28,224665,None,None,NaT,...,,[],[],SemVer18,NaN,NaN,2019,Nov,2019-11-20 06:03:28,06:03:28
67227,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,2019-11-15 02:50:28,224665,None,None,NaT,...,,[],[],SemVer20,NaN,NaN,2019,Nov,2019-11-15 02:50:28,02:50:28
67228,224665,alb-sdk,vmware,TestSeDatastoreLevel2.yaml,28,2019-10-01 08:28:36,224665,None,None,NaT,...,,[],[],SemVer18,NaN,NaN,2019,Oct,2019-10-01 08:28:36,08:28:36
